In [1]:
!git clone https://github.com/ShathaTm/LK-Hadith-Corpus.git


'git' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import glob
path = '/content/LK-Hadith-Corpus'
files = sorted(glob.glob(path+'//**//*.csv', recursive=True))

In [3]:
import re

def clean_text(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  text = re.sub(r'\s+', ' ', text)
  return text

In [4]:
columns = [
    'Chapter_Number', 'Chapter_English', 'Chapter_Arabic',
    'Hadith_Number', 'English_Hadith', 'English_Isnad', 'English_Matn', 'English_Grade',
    'Arabic_Hadith', 'Arabic_Isnad', 'Arabic_Matn', 'Arabic_Grade'
]

In [5]:
import pandas as pd
columns.append('Cleaned_Hadith')
all_hadith = []
for f in files:
  df = pd.read_csv(f)
  df['Cleaned_Hadith'] = df['English_Hadith'].apply(clean_text)
  all_hadith.append(df[['Chapter_Number', 'Chapter_English']])

In [ ]:
final_df = pd.concat(all_hadith, ignore_index=True)

In [7]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\HP
[nltk_data]     LAPTOP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
final_df['Tokens'] = final_df['Cleaned_Hadith'].apply(word_tokenize)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(final_df['Cleaned_Hadith'].fillna(''))

In [ ]:
import numpy as np
cosine_sim = (tfidf_matrix * tfidf_matrix.T).toarray()

In [ ]:
def search_hadith(query):
    cleaned_query = clean_text(query)
    query_vec = tfidf.transform([cleaned_query])
    sim = (query_vec * tfidf_matrix.T).toarray()[0]
    idx = np.argsort(sim)[-5:][::-1]
    return final_df.iloc[idx]

In [ ]:
user_q = "charity in islam"
results = search_hadith(user_q)
results

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix)

In [8]:
import numpy as np
def get_top_matches(query, k=5):
    cleaned_query = clean_text(query)
    q_vec = tfidf.transform([cleaned_query])
    sim_scores = cosine_similarity(q_vec, tfidf_matrix).flatten()
    top_k = np.argsort(sim_scores)[-k:][::-1]
    return final_df.iloc[top_k]

In [9]:
get_top_matches("prayer")

NameError: name 'tfidf' is not defined

In [10]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

ModuleNotFoundError: No module named 'torch'

In [ ]:
def embed(text):
    tokens = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        output = model(**tokens)
    return output.last_hidden_state.mean(dim=1).squeeze()

In [ ]:
embeddings = torch.stack([embed(x) for x in final_df['Cleaned_Hadith'].fillna('')])

In [ ]:
def search_embed(query, k=5):
    q_emb = embed(clean_text(query))
    sims = torch.nn.functional.cosine_similarity(q_emb.unsqueeze(0), embeddings)
    top_k = torch.topk(sims, k).indices
    return final_df.iloc[top_k]

In [ ]:
search_embed("prophet muhammad's kindness")